# Hypertune cifar10

In [11]:
DATADIR = ("data/raw/cifar10/")

In [12]:
from torchvision import datasets, transforms
from loguru import logger
from pathlib import Path

# Ensure the data directory exists before downloading dataset
data_dir = Path(DATADIR).resolve()
if not data_dir.exists():
    data_dir.mkdir(parents=True)
    logger.info(f"Created {data_dir}")

# Create transformer to convert images to tensors
transformer = transforms.Compose([transforms.ToTensor()])

# Download CIFAR10 dataset
train_dataset = datasets.CIFAR10(root=data_dir, train=True, download=True, transform=transformer)
test_dataset = datasets.CIFAR10(root=data_dir, train=False, download=True, transform=transformer)

logger.info(
    f"Dataset is now available:\n"
    f"TRAIN: {train_dataset}\n"
    f"TEST: {test_dataset}"
)

2025-10-08 20:01:37.808 | INFO     | __main__:<module>:18 - Dataset is now available:
TRAIN: Dataset CIFAR10
    Number of datapoints: 50000
    Root location: /mnt/WORKSPACE/kiei_workspace/master/UOS3/portfolio-DLDEPL01_2025/4-hypertuning-ray/data/raw/cifar10
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
           )
TEST: Dataset CIFAR10
    Number of datapoints: 10000
    Root location: /mnt/WORKSPACE/kiei_workspace/master/UOS3/portfolio-DLDEPL01_2025/4-hypertuning-ray/data/raw/cifar10
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
           )


In [13]:
train_dataset.classes

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [14]:
from torch.utils.data import DataLoader

# Create data loaders for training and testing
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Inspect the shape of a batch of training data
for images, labels in train_loader:
    logger.info(f"Image batch dimensions: {images.shape}")
    logger.info(f"Image label dimensions: {labels.shape}")
    break

2025-10-08 20:01:37.834 | INFO     | __main__:<module>:9 - Image batch dimensions: torch.Size([64, 3, 32, 32])
2025-10-08 20:01:37.834 | INFO     | __main__:<module>:10 - Image label dimensions: torch.Size([64])


In [27]:
import torch
from mltrainer import metrics


config = {
    # Fixed parameters
    "epochs": 5,
    "data_dir": data_dir,
    "batch_size": 64,
    "input_size": 3,
    "output_size": 20,
    "hidden_size": 128,
    "dropout": 0,
    "num_layers": 3,
    "learning_rate": 0.001,
    "loss_fn": torch.nn.CrossEntropyLoss(), # suitable for multi-class classification
    "optimizer": torch.optim.Adam,
    # "scheduler": torch.optim.lr_scheduler.LRScheduler,
    "scheduler": torch.optim.lr_scheduler.ReduceLROnPlateau,
    "metrics" : metrics.Accuracy(), # Accuracy metric for classification
}


In [16]:
# Setup for simple neural network

import torch.nn as nn
from torchsummary import summary

class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout, num_layers):
        super(SimpleNN, self).__init__()
        self.input_size = input_size

        layers = []
        layers.append(nn.Flatten())
        layers.append(nn.Linear(input_size * 32 * 32, hidden_size))
        layers.append(nn.ReLU())
        layers.append(nn.Dropout(dropout))
        
        for _ in range(num_layers - 1):
            layers.append(nn.Linear(hidden_size, hidden_size))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
        
        layers.append(nn.Linear(hidden_size, output_size))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x) 

    def summary(self):
        summary(self.network, (self.input_size, 32, 32))

model_NN = SimpleNN(
    input_size=config["input_size"],
    hidden_size=config["hidden_size"],
    output_size=config["output_size"],
    dropout=config["dropout"],
    num_layers=config["num_layers"]
)

# Show a summary of the model architecture
model_NN.summary(); model_NN.summary

Layer (type:depth-idx)                   Output Shape              Param #
├─Flatten: 1-1                           [-1, 3072]                --
├─Linear: 1-2                            [-1, 128]                 393,344
├─ReLU: 1-3                              [-1, 128]                 --
├─Dropout: 1-4                           [-1, 128]                 --
├─Linear: 1-5                            [-1, 128]                 16,512
├─ReLU: 1-6                              [-1, 128]                 --
├─Dropout: 1-7                           [-1, 128]                 --
├─Linear: 1-8                            [-1, 128]                 16,512
├─ReLU: 1-9                              [-1, 128]                 --
├─Dropout: 1-10                          [-1, 128]                 --
├─Linear: 1-11                           [-1, 20]                  2,580
Total params: 428,948
Trainable params: 428,948
Non-trainable params: 0
Total mult-adds (M): 0.43
Input size (MB): 0.01
Forward/backward pa

<bound method SimpleNN.summary of SimpleNN(
  (network): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3072, out_features=128, bias=True)
    (2): ReLU()
    (3): Dropout(p=0, inplace=False)
    (4): Linear(in_features=128, out_features=128, bias=True)
    (5): ReLU()
    (6): Dropout(p=0, inplace=False)
    (7): Linear(in_features=128, out_features=128, bias=True)
    (8): ReLU()
    (9): Dropout(p=0, inplace=False)
    (10): Linear(in_features=128, out_features=20, bias=True)
  )
)>

In [33]:
# Train the model

import torch.optim as optim
import torch.nn.functional as F
import torch

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_NN.to(device)

# Train model_NN
optimizer = config["optimizer"](model_NN.parameters(), lr=config["learning_rate"])
loss_fn = config["loss_fn"]
num_epochs = config["epochs"]

train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=config["batch_size"], shuffle=False)
for epoch in range(num_epochs):
    model_NN.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model_NN(images)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    # Calculate average loss over an epoch
    avg_loss = running_loss / len(train_loader)
    # Calculate test loss after each epoch
    model_NN.eval()
    test_loss = 0.0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model_NN(images)
            loss = loss_fn(outputs, labels)
            test_loss += loss.item()
    test_loss /= len(test_loader)

    logger.info(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_loss:.4f}, Test Loss: {test_loss:.4f}, Images Analysed: {(epoch+1)*len(train_loader.dataset)}")

# Evaluate the model
model_NN.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model_NN(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    logger.info(f"Test Accuracy: {accuracy:.2f}%")



2025-10-08 20:14:18.140 | INFO     | __main__:<module>:45 - Epoch [1/5], Train Loss: 1.1860, Test Loss: 1.4413, Images Analysed: 50000
2025-10-08 20:14:21.370 | INFO     | __main__:<module>:45 - Epoch [2/5], Train Loss: 1.1711, Test Loss: 1.4196, Images Analysed: 100000
2025-10-08 20:14:24.595 | INFO     | __main__:<module>:45 - Epoch [3/5], Train Loss: 1.1601, Test Loss: 1.4098, Images Analysed: 150000
2025-10-08 20:14:27.810 | INFO     | __main__:<module>:45 - Epoch [4/5], Train Loss: 1.1512, Test Loss: 1.4430, Images Analysed: 200000
2025-10-08 20:14:31.037 | INFO     | __main__:<module>:45 - Epoch [5/5], Train Loss: 1.1368, Test Loss: 1.4280, Images Analysed: 250000
2025-10-08 20:14:31.432 | INFO     | __main__:<module>:59 - Test Accuracy: 50.55%


In [30]:
from mltrainer import Trainer, ReportTypes, TrainerSettings

model_NN2 = SimpleNN(
    input_size=config["input_size"],
    hidden_size=config["hidden_size"],
    output_size=config["output_size"],
    dropout=config["dropout"],
    num_layers=config["num_layers"]
)

model_NN2.to(device)

trainer = Trainer(
    model=model_NN2,
    settings=TrainerSettings(
        epochs=config["epochs"],
        metrics=[config["metrics"]],
        logdir=Path("./logs"),
        train_steps=len(train_loader),
        valid_steps=len(test_loader),
        reporttypes=[ReportTypes.TOML],
        scheduler_kwargs={"patience": 5},
        earlystop_kwargs={"patience": 5},
    ),
    loss_fn=config["loss_fn"],
    optimizer=torch.optim.Adam,
    traindataloader=train_loader,
    validdataloader=test_loader,
    scheduler=config["scheduler"],
    device=device,
)
trainer.loop()

2025-10-08 20:07:34.184 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to logs/20251008-200734
2025-10-08 20:07:34.190 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 782/782 [00:07<00:00, 106.78it/s]
2025-10-08 20:07:42.741 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 1.8887 test 1.6531 metric ['0.3281']
100%|██████████| 782/782 [00:09<00:00, 84.14it/s]
2025-10-08 20:07:52.831 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 1.6813 test 1.5538 metric ['0.4375']
100%|██████████| 782/782 [00:15<00:00, 51.17it/s]
2025-10-08 20:08:10.456 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 1.5952 test 1.5232 metric ['0.4219']
100%|██████████| 782/782 [00:19<00:00, 41.07it/s]
2025-10-08 20:08:32.030 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 1.5304 test 1.4781 metric ['0.4375']
100%|██████████| 782/782 [00:20<00:00, 38.72it/s]
2025-10-08 20:0